In [12]:
import numpy as np
from pathlib import Path
import re
from tqdm import tqdm
from skimage import io
from skimage.morphology import skeletonize_3d, binary_dilation
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes 
import cc3d
from scipy.io import loadmat, savemat
import skan
import networkx as nx
from collections import Counter
import pandas as pd

In [13]:
directory = Path('james_preds')
files  = directory.glob('*?[0-9]_warped.pickle')
files = sorted([x.as_posix() for x in files])
files = [x for x in files if 'clean' not in x]
files[0]

'james_preds/XYZres006_0001_warped.pickle'

In [4]:
for file in tqdm(files):
    graph = nx.read_gpickle(file)
    skel_dst = io.imread(re.sub('_warped.pickle','_dst_skel_warped.tif',file))
    for i in range(len(graph.edges)):
        path = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['pts']
        _vals = skel_dst[path[:,0],path[:,1],path[:,2]]
        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii'] = np.mean(_vals)
        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_std'] = np.std(_vals)
        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights'] = _vals
        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight'] = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight'] * 0.99436
        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0'] = path[0]
        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1'] = path[-1]
    nx.write_gpickle(graph, re.sub('.pickle','_radii.pickle',file))

100%|██████████| 94/94 [19:11<00:00, 12.25s/it]


In [17]:
directory = Path('james_preds')
files  = directory.glob('*_0001_warped_radii_clean.pickle')
files = sorted([x.as_posix() for x in files])

In [18]:
files[0]

'james_preds/XYZres006_0001_warped_radii_clean.pickle'

In [21]:
for file_0001 in tqdm(files):
    file = re.sub('_0001','',file_0001)
    graph = nx.read_gpickle(file)
    graph_0001 = nx.read_gpickle(file_0001)
    res = []
    thresh = 20
    count=0
    for i in list(graph.edges):
        for j in list(graph_0001.edges):
            if graph.edges[i]['weight'] > 5:
                thresh_dst = graph.edges[i]['weight']/2
                start = graph.edges[i]['end-0']
                end = graph.edges[i]['end-1']
                start_0001 = graph_0001.edges[j]['end-0']
                end_0001 = graph_0001.edges[j]['end-1']
                dst_start_mtch = np.sqrt(np.sum(np.square(start-start_0001)))
                dst_end_mtch = np.sqrt(np.sum(np.square(end-end_0001)))
                dst_start_opp = np.sqrt(np.sum(np.square(start-end_0001)))
                dst_end_opp = np.sqrt(np.sum(np.square(end-start_0001)))
                path_dst = graph.edges[i]['weight']
                path_dst_0001 = graph_0001.edges[j]['weight']
                distance_delt = abs(path_dst - path_dst_0001)
                if (dst_start_mtch < thresh and dst_end_mtch < thresh and distance_delt < thresh_dst or 
                    dst_start_opp < thresh and dst_end_opp < thresh and distance_delt < thresh_dst):
                    res.append([i,j])
                    graph.edges[i]['radii_0001'] = graph_0001.edges[j]['radii']
                    graph.edges[i]['radii_0001_std'] = graph_0001.edges[j]['radii_std']
                    break
                else:
                    continue
    res = np.array(res)
    
    if len(res) !=0:
        print(res.shape)
        res_str = [",".join(item) for item in res[:,1].astype(str)]
        repeated = [item for item, count in Counter(res_str).items() if count > 1]
        res = np.array(res)
        if len(repeated) != 0:
            for i in range(len(repeated)):   # branches_selected.to_excel(re.sub('.tiff','_iso.xlsx',file))
                res = np.delete(res,np.where(np.array(res_str) == repeated[i])[0],axis=0)
                res_str = np.delete(res_str,np.where(np.array(res_str) == repeated[i])[0],axis=0)
        res = np.array(res)
        print(res.shape)
        np.save(re.sub('.pickle','_edge_match.npy',file),res)
        nx.write_gpickle(graph, re.sub('_clean.pickle','_matched.pickle',file))
        print(re.sub('_clean.pickle','_matched.pickle',file))

  0%|          | 0/47 [00:00<?, ?it/s]/tmp/ipykernel_390016/1647967039.py:16: RuntimeWarning: invalid value encountered in sqrt
  dst_start_mtch = np.sqrt(np.sum(np.square(start-start_0001)))
/tmp/ipykernel_390016/1647967039.py:19: RuntimeWarning: invalid value encountered in sqrt
  dst_end_opp = np.sqrt(np.sum(np.square(end-start_0001)))
/tmp/ipykernel_390016/1647967039.py:17: RuntimeWarning: invalid value encountered in sqrt
  dst_end_mtch = np.sqrt(np.sum(np.square(end-end_0001)))
/tmp/ipykernel_390016/1647967039.py:18: RuntimeWarning: invalid value encountered in sqrt
  dst_start_opp = np.sqrt(np.sum(np.square(start-end_0001)))
  2%|▏         | 1/47 [00:04<03:31,  4.59s/it]

(37, 2, 2)
(31, 2, 2)
james_preds/XYZres006_warped_radii_matched.pickle


  4%|▍         | 2/47 [00:07<02:43,  3.64s/it]

(152, 2, 2)
(117, 2, 2)
james_preds/XYZres007_warped_radii_matched.pickle


  6%|▋         | 3/47 [00:12<03:06,  4.23s/it]

...

(355, 2, 2)
(265, 2, 2)
james_preds/XYZres050_warped_radii_matched.pickle


 98%|█████████▊| 46/47 [03:36<00:07,  7.25s/it]

(363, 2, 2)
(281, 2, 2)
james_preds/XYZres051_warped_radii_matched.pickle


100%|██████████| 47/47 [03:38<00:00,  4.65s/it]

(187, 2, 2)
(155, 2, 2)
james_preds/XYZres052_warped_radii_matched.pickle


In [ ]:
print(res.shape)

In [ ]:
res_str = [",".join(item) for item in res[:,1].astype(str)]
repeated = [item for item, count in Counter(res_str).items() if count > 1]

In [ ]:
res_str = [",".join(item) for item in res[:,1].astype(str)]
repeated = [item for item, count in Counter(res_str).items() if count > 1]

res = np.array(res)
if len(repeated) != 0:
    for i in range(len(repeated)):   # branches_selected.to_excel(re.sub('.tiff','_iso.xlsx',file))
# branches_selected_0001.to_excel(re.sub('.tiff','_iso.xlsx',file_0001))
# branches_full.to_excel(re.sub('.tiff','_iso_full.xlsx',file))
# branches_full_0001.to_excel(re.sub('.tiff','_iso_full.xlsx',file_0001))
# np.save(re.sub('.tiff','_iso_corr.txt',file),res)
        res = np.delete(res,np.where(np.array(res_str) == repeated[i])[0],axis=0)



res = np.array(res)
print(res.shape)

In [11]:
file = files[1]

In [12]:
graph = nx.read_gpickle(file)

In [21]:
graph[82][121]

{'pts': array([[ 67,  12, 152],
        [ 67,  13, 152],
        [ 67,  14, 152],
        [ 67,  15, 152],
        [ 67,  16, 152],
        [ 67,  17, 152],
        [ 67,  18, 152],
        [ 68,  19, 152],
        [ 68,  20, 152],
        [ 68,  21, 151],
        [ 68,  22, 150],
        [ 68,  23, 149],
        [ 68,  24, 149],
        [ 68,  25, 148],
        [ 68,  26, 147],
        [ 67,  27, 147],
        [ 67,  28, 147],
        [ 67,  29, 147],
        [ 68,  30, 146],
        [ 68,  31, 145],
        [ 69,  32, 144],
        [ 69,  33, 144],
        [ 69,  34, 143],
        [ 69,  35, 143],
        [ 69,  36, 142],
        [ 69,  37, 141],
        [ 70,  38, 140],
        [ 70,  38, 139],
        [ 70,  39, 138],
        [ 71,  40, 137],
        [ 71,  41, 136],
        [ 72,  42, 136],
        [ 73,  43, 135],
        [ 73,  44, 134],
        [ 74,  45, 133],
        [ 74,  46, 133],
        [ 75,  47, 132],
        [ 75,  48, 132],
        [ 76,  49, 131],
        [ 76,  50,

In [16]:
list(graph.edges)

[(0, 5),
 (1, 46),
 (2, 3),
 (6, 15),
 (7, 9),
 (8, 12),
 (10, 70),
 (11, 42),
 (13, 26),
 (14, 78),
 (16, 63),
 (17, 31),
 (18, 19),
 (20, 34),
 (21, 24),
 (23, 30),
 (27, 39),
 (28, 42),
 (29, 35),
 (31, 37),
 (31, 36),
 (32, 47),
 (33, 38),
 (38, 45),
 (40, 60),
 (41, 48),
 (42, 112),
 (50, 61),
 (51, 54),
 (52, 91),
 (53, 56),
 (55, 62),
 (58, 78),
 (59, 62),
 (60, 93),
 (60, 75),
 (62, 124),
 (64, 69),
 (65, 68),
 (66, 90),
 (67, 106),
 (72, 74),
 (73, 86),
 (75, 213),
 (75, 100),
 (77, 88),
 (78, 91),
 (80, 81),
 (82, 121),
 (83, 84),
 (88, 88),
 (88, 99),
 (89, 96),
 (91, 206),
 (94, 98),
 (95, 97),
 (101, 116),
 (102, 103),
 (102, 119),
 (102, 104),
 (105, 166),
 (109, 115),
 (111, 132),
 (112, 118),
 (112, 128),
 (113, 146),
 (114, 126),
 (117, 126),
 (122, 123),
 (124, 152),
 (124, 216),
 (125, 143),
 (126, 167),
 (127, 131),
 (127, 127),
 (127, 159),
 (127, 137),
 (128, 129),
 (128, 163),
 (130, 212),
 (133, 135),
 (134, 135),
 (135, 154),
 (136, 150),
 (138, 173),
 (140, 15